# Goal: 
To obtain a txt file for each work with metadata as: title and year of publication.

## Download the pickel pre-corpus

In [1]:
import pickle
import re

In [2]:
corpus= pickle.load( open( "corpus.pickle", "rb" ) )

## Remove unwanted characters

* °syl-lables
* ae
* pages

In [3]:
patterns=[("°(.*)\n",'\n'),("æ",'ae'),("Æ",'Ae'),("―(\s[0-9]+\s)―"," ")]
clean_corpus=[]

for title,text in corpus:
    for old,new in patterns:
        text=re.sub(old,new,text)
    clean_corpus.append([title,text])

In [11]:
import pickle
pickle.dump( clean_corpus, open( "clean_corpus.pickle", "wb" ) )

# Split and assign dates

The goal is to assign to each work the year of pubblication.

final_corpus =\[ {"title": title, "year", year_of_pubblication, "content": text} , ... \]

Some works are already isolated, other are not.

* understand which works are isolated and which are not
* understand how to split the ones with more works inside

#### Considerations: 
it doesn't matter for the aim of this project to really have a precise split for each work, what really matter is to have the correct assignment between text and year. Especially considering that only secondary works are not already splitted. More than that, it is worth noting that the year of pubblicashion is just a proxy of the period in which a work was written; perhaps earlier works were published later due to pubblicashion issues, but for a such large corpus this should not be a problem. Later on may be interesting to ask to some expertise if some work have a longer period of writting with respect to others and see if it have some impact or can be detected by the analysis.

In [12]:
corpus= pickle.load( open( "clean_corpus.pickle", "rb" ) )

In [13]:
titles=[x for x,y in corpus]

In [14]:
len(titles)

174

In [15]:
titles[10]

'The Early Works of John Dewey, 1882-1898. Volume 4: 1893-1894, Essays, The Study of Ethics : THE STUDY OF ETHICS '

In [16]:
titles_indexed=[(x,corpus.index([x,y])) for x,y in corpus]
titles_indexed[:5]

[("The Early Works of John Dewey, 1882-1898. Volume 1: 1882-1888, Essays, Leibniz's New Essays Concerning the Human Understanding : EARLY ESSAYS ",
  0),
 ("The Early Works of John Dewey, 1882-1898. Volume 1: 1882-1888, Essays, Leibniz's New Essays Concerning the Human Understanding : LEIBNIZ'S NEW ESSAYS CONCERNING THE HUMAN UNDERSTANDING  ",
  1),
 ('The Early Works of John Dewey, 1882-1898. Volume 2: 1887, Psychology : PSYCHOLOGY   ',
  2),
 ('The Early Works of John Dewey, 1882-1898. Volume 3: 1889-1892, Essays, Outlines of a Critical Theory of Ethics : EARLY ESSAYS ',
  3),
 ('The Early Works of John Dewey, 1882-1898. Volume 3: 1889-1892, Essays, Outlines of a Critical Theory of Ethics : BOOK REVIEWS ',
  4)]

In [17]:
#Trivial information retrieval system, search works per words present in the title
title="STUDIES IN LOGICAL THEORY"
indexes=[corpus.index([x,y]) for x,y in corpus if title in x]
indexes

[27]

In [18]:
#given the index take the info
def info(i):
    volume=titles_indexed[i][0].split(':')[0].strip()
    work=titles_indexed[i][0].split(':')[2].strip().capitalize()
    date=titles_indexed[i][0].split(':')[1].split(',',1)[0].strip()
    information={'Volume: ':volume,"Title: ":work,'Date: ':date, 'Index':i}
    return information

In [19]:
info(1)

{'Volume: ': 'The Early Works of John Dewey, 1882-1898. Volume 1',
 'Title: ': "Leibniz's new essays concerning the human understanding",
 'Date: ': '1882-1888',
 'Index': 1}

In [20]:
infos=[info(i) for i in range(len(corpus))]

In [23]:
infos[:3]

[{'Volume: ': 'The Early Works of John Dewey, 1882-1898. Volume 1',
  'Title: ': 'Early essays',
  'Date: ': '1882-1888',
  'Index': 0},
 {'Volume: ': 'The Early Works of John Dewey, 1882-1898. Volume 1',
  'Title: ': "Leibniz's new essays concerning the human understanding",
  'Date: ': '1882-1888',
  'Index': 1},
 {'Volume: ': 'The Early Works of John Dewey, 1882-1898. Volume 2',
  'Title: ': 'Psychology',
  'Date: ': '1887',
  'Index': 2}]

### Testing the regex pattern matching and splitting

Not many works have unique year, but what if I cannot trust the date indicated by titles
Should I really check any text?

Please note the use of \[ as a essay delimiter, but not only

Or maybe better, [First pubblished ... ]

\[First published(.|\n[^»*])*

### Splitting function and testing its behaviuor

Need a function that just take a text and split in sublist of (title, info of first pubblication, text)

In [26]:
def needed_split(text):
    x=re.split("(\[(.[^»*])*First published(.|\n[^»*])*)",text, maxsplit=10000000) #split the recurrent pattern of infos
    x=[X for X in x if X!=']'and X!='. 'and X!=None] #get rid of useless elements
    y=list()
    if len(x)>1: #at least one occurence of the pattern
        for i in range(len(x)):
            if i%2==0and i>1: #if it is a text + title
                z=re.split("((.+)$)",x[i])
                y.append(z[0])
                y.append(z[1].strip())
            else: #otherwise is already isolated
                y.append(x[i])
        return y[:-1] #get rid of the last useless element
    else: #zero occurence of the pattern
        y=x
        return y

In [27]:
splitted=needed_split(corpus[11][1]) #1, 6, 7, 10

In [28]:
len(splitted)

6

In [29]:
print(splitted[0])

Appendixes

  
                            
  
                            
1: The Relation of Philosophy to Theology 


In [30]:
print(splitted[1])

[First published in the Monthly Bulletin of the Students' Christian
Association of the University of Michigan, XVI (Jan. 1893), 66-
68. Not previously reprinted.]


In [32]:
print(splitted[2][:500])


»*
In the latter part of the last college year Professor Dewey
addressed the Ministerial Band on the above subject. It was
expected that a stenographic report would be made, but as this
has not been done, the readers of the Bulletin may find of value
the following summary from notes:
In discussing this question I wish you to remember
that I take the standpoint of philosophy. The theologians
will be able to take care of their end. What is philosophy?
The value of philosophy lies in its method, n


In [33]:
print(splitted[3])

3: Ethics and Politics


In [ ]:
print(splitted[4])

In [ ]:
print(splitted[5])

In [ ]:
print(splitted[-1])

In [ ]:
print(splitted[-2])

In [ ]:
print(splitted[-3])

In [ ]:
print(splitted[-4])

### One possible solution

#### Define a function that use the needed_split() to obtain a raw final corpus

[volume title: ... , title: ... , pubblication infos: ... , text: ,,, ]

#### Problem:

single text have a different format

#### Solution:

add a new level..

[{'volume title': ... , content: [text]} , {'volume title': ... , content: [(title, info, text), ... ]}]

#### Obs:

This structure is information keeper but over articolated

### KISS SOLUTION

#### Just obtain the corpus that I need

* extract the year
* stay faithful to: final_corpus =\[ {"title": title, "year", year_of_pubblication, "content": text} , ... \]
* title may be a concatenation of volume and real title

#### extracting the year experiment

* first case: a unique text
* second case: set of essays

In [34]:
#first case
splitted=needed_split(corpus[10][1])
len(splitted)

1

In [40]:
print(splitted[0][:500])

The Study of Ethics
THE STUDY OF ETHICS
 

 
    A SYLLABUS
 

 
    BY JOHN DEWEY
 
ANN ARBOR MICHIGAN
register publishing company
The Inland Press
1894

  
                            
  
                            
Prefatory Note

THE EDITION of my Outlines of Ethics [Early Works, III, 237-
388] having been exhausted, I have prepared the following
pages, primarily for the use and guidance of my own students.
The demand for the former book seems, however, to justify
the belief that, amid the 


In [41]:
date=re.findall('(\d{4})', splitted[0])
print(date)
#take the first date

['1894', '1880', '1872', '1748', '1832', '1773', '1836', '1806', '1873', '1888', '1894']


In [42]:
#second case
splitted=needed_split(corpus[2][1])
len(splitted)

3

In [43]:
date=re.findall('(\d{4})', splitted[1])
print(date)
#take the last date, does exists other strange cases other than the first essays of the eighth work?

['1887']


## Finally

In [45]:
final_corpus=list()
for j in range(len(corpus)):
    general_title=corpus[j][0]
    content=corpus[j][1]
    list_of_works=needed_split(content)
    if len(list_of_works)==1:
        title=infos[j]['Volume: ']+' / '+infos[j]['Title: ']
        dates=re.findall('(\d{4})', list_of_works[0])
        for ye in dates:
            d=int(ye)
            if d>=1870 and d<=1952:
                date=d
                break
        text=content
        final_corpus.append({'title: ': title, 'year: ': date, 'text: ': text})
    elif len(list_of_works)>1:
        for i in range(0,len(list_of_works)-1,3): # i is the index of the title
            title=infos[j]['Volume: ']+' / '+infos[j]['Title: ']+' / '+list_of_works[i]
            dates=re.findall('(\d{4})', list_of_works[i+1])
            if len(dates)>0:
                for ye in dates:
                    d=int(ye)
                    if d>=1870 and d<=1952:
                        date=d
                        break
            else: #if no year is clearly indicated then take the year from the general title as a proxy, STUDIES IN LOGICAL THEORY  3,6,9 
                date=infos[j]['Date: ']
            text=list_of_works[i+2]
            final_corpus.append({'title: ': title, 'year: ': date, 'text: ': text})

In [46]:
len(final_corpus)

1095

In [48]:
final_corpus[144]

{'title: ': 'The Middle Works of John Dewey, 1899-1924. Volume 2 / Studies in logical theory / 4. The Objects of Thought',
 'year: ': '1902-1903',
 'text: ': '\n»*\nIn the foregoing discussion, particularly in the last\nchapter, we were repeatedly led to recognize that thought\nhas its own distinctive objects. At times Lotze gives way to\nthe tendency to define thought entirely in terms of modes\nand forms of activity which are exercised by it upon a strictly\nforeign material. But two motives continually push him in\nthe other direction. (1) Thought has a distinctive work to\ndo, one which involves a qualitative transformation of (at\nleast) the relationships of the presented matter; as fast as it\naccomplishes this work, the subject-matter becomes \nsomehow thought\'s subject-matter. As we have just seen, the data\nare progressively organized to meet thought\'s ideal of a \ncomplete whole, with its members interconnected according to\na determining principle. Such progressive organiz

In [50]:
print(final_corpus[144]['text: '][:400])


»*
In the foregoing discussion, particularly in the last
chapter, we were repeatedly led to recognize that thought
has its own distinctive objects. At times Lotze gives way to
the tendency to define thought entirely in terms of modes
and forms of activity which are exercised by it upon a strictly
foreign material. But two motives continually push him in
the other direction. (1) Thought has a dist


In [51]:
#clean from »*
patterns=[("[»*]",'')]
for work in final_corpus:
    for old,new in patterns:
        work['text: ']=re.sub(old,new,work['text: '])

In [52]:
pickle.dump( final_corpus, open( "splitted_corpus.pickle", "wb" ) )

### Problems:

* use of a not completely safe heuristics
* how to solve the range of years problem?

In [53]:
final_corpus= pickle.load( open( "splitted_corpus.pickle", "rb" ) )

In [54]:
final_corpus[1035]

{'title: ': 'The Later Works of John Dewey, 1925-1953. Volume 17 / Reviews / The Promise of American Politics',
 'year: ': 1937,
 'text: ': '\n\nBy T. V. Smith. Chicago: University of Chicago Press, 1936.\nLet me say at the outset that Dr. Smith\'s book is an\ninterpretation of the actualities and possibilities of American politics\nin moral terms, based upon sound psychology. But when I say\nmoral, I do not mean moralistic—anything but that. He keeps\nhis eye steadily upon human relations and associations and what\nthey do and may do to make the lives of individuals who are\ncaught up in these associations full and significant, for\nthemselves and for others. This is the only meaning I can attach to\n"moral." The book displays imagination, but it is an imagination\nso based upon the realities of our common American life that\nimagination never degenerates into fantasy. The style in which\nthe book is written accords with its substance; it is animated as\nwell as lucid. The liveliness 

In [55]:
final_corpus[173]

{'title: ': 'The Middle Works of John Dewey, 1899-1924. Volume 3 / Reviews / World Views and Their Ethical Implications',
 'year: ': 1904,
 'text: ': '\n: A Syllabus of Lectures in Advanced Ethics.\n\nBy W. R. Benedict, Professor of Philosophy in the University of Cincinnati.Cincinnati University Press, 1902.\nI am glad to have an opportunity of calling attention to\nthis little volume of a hundred pages, prepared as a syllabus\naccompanying collegiate lectures. No one would doubt for a\nmoment that all systems of philosophy, or World Views,\ncarry with them implications of the utmost significance for\nconduct; few would deny that one of the main motifs in the\ngeneration of philosophic systems has been the consciousness\nof certain ultimate issues as to morality, the possibility of\nrationally valuing life as worthy or unworthy. Such being\nthe case, it is a striking fact that there is almost a total lack\nof literature upon the inherent bearings of philosophic \nstandpoints and outlo

In [56]:
# ad hoc solution
for i in range(141,145):
    final_corpus[i]['year: ']='1903'

In [57]:
final_corpus.pop(171)

{'title: ': 'The Middle Works of John Dewey, 1899-1924. Volume 3 / Essays / [',
 'year: ': '1903-1906',
 'text: ': '[First published in The Psychology of Child Development by\nIrving W. King (Chicago: University of Chicago Press, 1903),\n'}

In [58]:
# ad hoc solution
for i in range(1035,1039):
    final_corpus[i]['year: ']='1918'

In [60]:
#problem solved
[(final_corpus.index(x),x['year: ']) for x in final_corpus if len(str(x['year: ']))!=4]

[]

# PROBLEMS 2.0

### YEARS DISTRIBUTION

In [61]:
years=[]
count=[]
for work in final_corpus:
    y=int(work['year: '])
    if y in years:
        count[years.index(y)]+=1
    else:
        years.append(y)
        count.append(1)
        
years_count=dict(zip(years,count))

In [62]:
years_count

{1882: 2,
 1883: 1,
 1884: 5,
 1885: 13,
 1886: 12,
 1887: 5,
 1888: 2,
 1889: 7,
 1890: 10,
 1891: 14,
 1892: 7,
 1893: 8,
 1894: 12,
 1897: 10,
 1896: 19,
 1898: 12,
 1895: 5,
 1899: 3,
 1900: 9,
 1901: 9,
 1902: 23,
 1903: 18,
 1904: 9,
 1906: 7,
 1905: 5,
 1909: 14,
 1908: 8,
 1907: 9,
 1912: 10,
 1910: 11,
 1911: 10,
 1913: 9,
 1914: 5,
 1915: 15,
 1942: 17,
 1916: 27,
 1917: 29,
 1918: 30,
 1919: 16,
 1920: 13,
 1948: 9,
 1922: 22,
 1921: 32,
 1924: 15,
 1923: 26,
 1925: 16,
 1926: 21,
 1927: 24,
 1946: 9,
 1928: 32,
 1943: 9,
 1929: 29,
 1930: 36,
 1941: 14,
 1931: 40,
 1932: 48,
 1933: 43,
 1934: 31,
 1935: 33,
 1936: 22,
 1937: 31,
 1938: 16,
 1939: 22,
 1940: 15,
 1944: 9,
 1945: 7,
 1949: 11,
 1947: 11,
 1950: 5,
 1951: 3,
 1952: 3}

## MISSED SPLIT

In [63]:
problems=[x['title: '] for x in final_corpus if len(x['title: '])>500]

In [64]:
len(problems)

13

In [65]:
problems[12]

'The Later Works of John Dewey, 1925-1953. Volume 17 / Unpublished writings / Unpublished Writings\nThe Historical Method in Ethics [Address delivered to the Philosophical Club, University of Chicago, 4 December\n1901. Typescript, not typed by Dewey, in the Joseph Ratner/John Dewey Papers,\nBox 44, folder 7, Special Collections, Morris Library, Southern Illinois University\nat Carbondale, and in the Henry Waldgrave Stuart Papers, Stanford University\nLibraries, Stanford, Calif.]\n»»\nThe remarks of my paper have arisen in connection with\nthe course I am giving at present, where I am dealing with the\nhistorical development of conduct, and which has led from this\nto a consideration of the possibility of a science of morality. I\nhave naturally arrived at the foregone conclusion that there is a\nscience of morality possible, and I have further concluded that\nthe condition of a scientific treatment of morality is in the use of\na universal method; in other words, that it is only by a h

In [66]:
problems[2]

'The Middle Works of John Dewey, 1899-1924. Volume 1 / The educational situation / The Educational Situation\n  \n                            \n  \n                            \nPrefatory Word\nIn the following paper I have attempted to set forth\nthe educational situation as it manifests itself in the three\ntypical parts of our educational system. In so doing, I have\nrevised papers originally prepared for three different bodies,\nnamely, the Superintendence section of the National \nEducational Association; the Conference of Secondary Schools \naffiliated with the University of Chicago; and the Harvard\nTeachers Association. If the following paper in the reading\nleaves with the reader the impression of a miscellaneous\ncollection, not of an organic unity, it will hardly be worth\nwhile for me here to iterate that it is an attempt to apply a\nsingle social philosophy, a single educational philosophy, to\na single problem manifested in forms that are only outwardly\ndiverse. I may, h

#### Some titles had ingested text due to pattern missing, as they seems to be an irrelevant part of the whole corpus, I just get rid of them

In [67]:
final_corpus=[x for x in final_corpus if len(x['title: '])<500]

In [68]:
pickle.dump( final_corpus, open( "splitted_corpus.pickle", "wb" ) )